In [1]:
import pandas as pd
import numpy as np
import math

In [11]:
df = pd.read_csv(r"sort_bug2.csv",sep=',')
df

,,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s82,s83,s84,s85,s86,s87,s88,s89,s90,r
0,t1,1,1,1,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
1,t2,1,1,1,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
2,t3,1,1,1,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
3,t4,1,1,1,1,1,0,1,0,1,...,1,1,1,0,1,1,1,1,1,0
4,t5,1,1,1,1,1,0,1,1,0,...,1,1,1,0,1,1,1,1,1,0
5,t6,1,1,1,1,1,0,1,1,0,...,1,1,1,0,1,1,1,1,1,0
6,t7,1,1,1,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
7,t8,1,1,1,1,1,0,1,0,1,...,1,1,1,0,1,1,1,1,1,0
8,t9,1,1,1,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
9,t10,1,1,1,1,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1


In [12]:
class Tarantula(object):

    def __init__(self, directory):

        self.__M = []  # boolean. coverage matrix
        self.__MT = []
        self.__F = []  # boolean. failing test cases
        self.__C = []
        self.__numOrigTests = 0
        self.__numStmts = 0
        self.__totalOrigFail = 0
        self.__totalOrigPass = 0
        self.__passOnStmt = []  # int. p(s), for every s and considering liveness of test cases
        self.__failOnStmt = []  # int. p(s), for every s and considering liveness of test cases
        self.__passRatio = []
        self.__failRatio = []
        self.__suspiciousness = []
        self.__directory = directory  # to store the coverage matrix file
        self.__ranking = []



    def readFile(self):
        self.__M = pd.read_csv(self.__directory, sep=',', index_col=0)

        # initialize all case are live
        self.TarantulaSuspiciousnessCalculation(self.__M)

        # initialize the fail and pass case number
        self.setF(self.__M.r)
        # print(self.__F)

    def TarantulaSuspiciousnessCalculation(self, M):
        self.__M = M

        self.__numOrigTests = self.__M.shape[0]
        # print(self.__numOrigTests)
        self.__numStmts = self.__M.shape[1] - 1
        # print(self.__numStmts)


    def setDirectort(self, dir):
        self.__directory = dir

    def setC(self, C):
        self.__C = C

    def setF(self, F):
        self.__F = F.tolist()
        self.calculateOrigFailAndPass()

    def getM(self):
        return self.__M

    def getF(self):
        return self.__F

    def getPassOnStmt(self):
        return self.__passOnStmt

    def getFailOnStmt(self):
        return self.__failOnStmt


    def mainCompute(self):
        self.calculatePassOnStmtAndFailOnStmt()
        self.calculatePassRatioAndFailRatio()
        self.calculateSuspiciousness()
        self.calculateRanking()

    def getSuspiciousness(self):
        return self.__suspiciousness

    def getOrigNumTests(self):
        return self.__numOrigTests

    def getOrigTotalFail(self):
        return self.__totalOrigFail

    def getOrigTotalPass(self):
        return self.__totalOrigPass

    def getNumStmts(self):
        return self.__numStmts

    def calculatePassOnStmtAndFailOnStmt(self):
        for i in range(self.__numStmts):
            self.__C = self.__M.iloc[:, i]
            self.__failOnStmt.insert(i, self.__C.tolist().count(0))
            self.__passOnStmt.insert(i, self.__C.tolist().count(1))

    def calculatePassRatioAndFailRatio(self):



        for i in range(self.__numStmts):
            if self.__totalOrigPass == 0:
                self.__passRatio.insert(i, 0)
            else:
                self.__passRatio.insert(i, self.__passOnStmt[i] / self.__totalOrigPass)

            if self.__totalOrigPass == 0:
                self.__failRatio.insert(i, 0)
            else:

                self.__failRatio.insert(i, self.__failOnStmt[i] / self.__totalOrigFail)

    def calculateSuspiciousness(self):
        for i in range(self.__numStmts):
            if (self.__totalOrigFail == 0) & (self.__totalOrigPass == 0):
                self.__suspiciousness.insert(i, -1)
            elif (self.__failRatio[i] == 0) & (self.__passRatio[i] == 0):
                self.__suspiciousness.insert(i, -1)
            else:
                self.__suspiciousness.insert(i, round(self.__failRatio[i] / (self.__failRatio[i] + self.__passRatio[i]), 2))

    def calculateOrigFailAndPass(self):
        for i in range(self.__numOrigTests):
            if self.__F[i]:

                self.__totalOrigPass += 1
            else:

                self.__totalOrigFail += 1

    def calculateRanking(self):
        # print whole list with suspiciousness and rank
        self.__MT = self.__M.T
        self.__suspiciousness.append(np.nan)
        self.__suspiciousness = pd.Series(self.__suspiciousness, index=[var for var in self.__M.columns])
        self.__ranking = self.__suspiciousness.rank(ascending=False, method="min")

        self.__MT["Suspiciousness"] = self.__suspiciousness
        self.__MT['Rank'] = self.__ranking
        self.__MT.sort_values("Rank", inplace=True)

        print(self.__MT)

In [13]:
if __name__ == "__main__":
    file = r"sort_bug2.csv"
    cal = Tarantula(file)
    cal.readFile()
    cal.mainCompute()

     t1  t2  t3  t4  t5  t6  t7  t8  t9  t10  ...   t13  t14  t15  t16  t17  \
s45   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s49   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s25   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s26   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s27   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s28   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s29   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s30   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s31   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s23   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s32   0   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s34   0   0   0   0   0   0   0   0   0    0  ...   